<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/W%26B_Training_with_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch + W&B

The purpose of this lab is to instrument W&B a top of existing ML Workflows which might be leveraging 
* PyTorch
* Tensorboard (for metric tracking)
* Python `logging` (for metric tracking)

We will augment this workflow by leveraging 
* Wandb Experiments and syncing with Tensorboard
* Wandb logging
* Wandb Artifacts for dataset and model logging / versioning
* Tables to surface prediction examples on Test datasets
* track lineage of all artifacts and experiments completed

Lastly, we'll do a simple demo of sweeps and interact with the runs via W&B API to
* query runs and run summaries
* artifacts

In [1]:
%%capture
!pip install wandb easydict optuna --upgrade

In [2]:
%%capture
!pip install tensorboard dill

## Logging In

In [3]:
#@title Enter host address
#@markdown Enter the host url which corresponds to your WB instance.
host = "https://api.wandb.ai" #@param {type: "string"}


In [4]:
import wandb
## when using wandb anywhere other than wandb.ai, you must 
## provide a proper host, so the client knows where to communcate
## details of the experiment
# wandb.login(key = key, host = host)
wandb.login(host = host)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import os
import random
import logging
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
# if you wind up with any device other than cpu, some code below will need to 
# change specific to the way we are interacting with torch tensors.  
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# remove slow mirror from list of MNIST mirrors
torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
                                      if not mirror.startswith("http://yann.lecun.com")]

## Get Data (and log it)

When we get data and log it, there are obviously tons of way to complete this.  Depending on how you log data, and whether or not you log your retrieval mechanism is a matter of preference and internal guidelines you need to follow.  

In our approach, we will write a `getter` for our data.  The benefit of writing a getting, we can log this getter with our dataset as part of the artifact metadata.



Before we get started it is important to set the name space for your project.  This is going to be accomplished by passing a `project_name` as well as an `entity` to your wandb experiment

`entity` corresponds to the team to which the project will be associated with.  The `entity` could be a team name, or your user name.  

In [6]:
project_name = "demos" #@param {type: "string"}
entity = "tim-w" #@param {type: "string"}

## Logging data

W&B is very unopinionated with regard to how you track your experiments.  We could log data in any number of ways.  
* Log one artifact which represents all the data - training, validation, and test data to one artifact 
* Log several artifacts - one for each of the training, validation, and test data loaders.  

It is a matter of what best suites your needs and workflows and expectations.  

### Anatomy of an artifact 

The `Artifact` class will correspond to an entry in the W&B Artifact registry.  The artifact has 
* a name
* a type
* metadata
* description
* files, directory of files, or references

Example usage 
```
run = wandb.init(project = "my-project")
artifact = wandb.Artifact(name = "my_artifact", type = "data")
artifact.add_file("/path/to/my/file.txt")
run.log_artifact(artifact)
run.finish()
```

In [21]:
## create the data directory locally if it does not already exists
from pathlib import Path
data_path = Path("./data")
data_path.mkdir(exist_ok = True)

## define out data getter 
def get_data(slice=5, train=True):
  '''
  helper function to get data
  args: 
    slice: Int => passed to torch.utils.data.Subset indices argument
    train: Boolean => True to download training data, False for test data
  '''
  full_dataset = torchvision.datasets.MNIST(root=".",
                                            train=train, 
                                            transform=transforms.ToTensor(),
                                            download=True)
  #  equiv to slicing with [::slice] 
  sub_dataset = torch.utils.data.Subset(
    full_dataset, indices=range(0, len(full_dataset), slice))

  return sub_dataset

In [22]:
logging.basicConfig(
                format="%(levelname)s - %(asctime)s - %(message)s",
        )
logger = logging.getLogger("CNN-Logger")
logger.setLevel("INFO")

## Our First W&B Experiment / Run

We are going to 
* get our training and test data
* split the training data into training and validation
* create artifacts for all three dataset
* log those artifacts to W&B.  

In [23]:
#%%wandb -h 600 
import pickle
from dill.source import getsource
from dill import detect
from datetime import datetime 

with wandb.init(project = project_name, job_type = "data-acquisition") as run:

  train, test = get_data(train=True), get_data(train=False)
  train, validation = torch.utils.data.random_split(train, [10000, 2000])

  torch.save(train, './data/training_data.pt')
  torch.save(validation, './data/validation_data.pt')
  torch.save(test, './data/test_data.pt')

  train_artifact = wandb.Artifact(name = "mnist-training-data", type = "dataset", 
                                  description = "training data",
                                  metadata = { 
                                      "data-set": "MNIST training",
                                      "getter": getsource(detect.code(get_data))}
                                  )
  train_artifact.add_file("./data/training_data.pt")

  validation_artifact = wandb.Artifact(name = "mnist-validation-data", type = "dataset", 
                                       description = "validation data",
                                       metadata = { 
                                      "data-set": "MNIST validation",
                                      "getter": getsource(detect.code(get_data))})
  validation_artifact.add_file("./data/validation_data.pt")

  test_artifact = wandb.Artifact(name = "mnist-test-data", type = "dataset", 
                                 description = "test data",
                                 metadata = { 
                                      "data-set": "MNIST test",
                                      "getter": getsource(detect.code(get_data))})
  test_artifact.add_file("./data/test_data.pt")  
  
  run.log_artifact(train_artifact)
  run.log_artifact(validation_artifact)
  run.log_artifact(test_artifact)

## Artifact usage (Creating the DAG)

Part of the value of W&B is the ability to capture lineage via Experiments and Artifacts.  Next up for our work flow is to specifiy a model and commence training.  

It is key to remember that experiments create and consume artifacts and we have already completed one  experimemtns where we created dataset artifacts.  

Next up, we will commence an experiment that will consume the artifacts from the previous run for the purposes of training model, then we will create a model artifact.

## Specify the model



In [24]:
# Conventional and convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [25]:
def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

## Training

In our first model training experiment, we are going to sync our wandb experiment to tensorboard -> so no wandb specific logging will be instrumented.  


In [26]:
# %%wandb -h 600
# Run training and track with wandb, but no explicit logging.  
# since we were alredy using tensorboard via WritterSumamry, we'll 
# sync w&b to tensorboard.
config = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.01,
    dataset="MNIST",
    architecture="CNN"
    )

with wandb.init(project = project_name, 
                 job_type = "training", 
                 config = config,
                 sync_tensorboard = True) as run:

  config = wandb.config
  ## or, ifyou have a nasty nested dictionary for your config
  # config = EasyDict(wandb.config)

  run.use_artifact(f"{run.entity}/{run.project}/mnist-training-data:latest")
  run.use_artifact(f"{run.entity}/{run.project}/mnist-validation-data:latest")
  ## download and instantiation of the artifacts might be necessary.  

  train_loader = make_loader(train, batch_size=config.batch_size)
  validation_loader = make_loader(validation, batch_size=config.batch_size)

  model = ConvNet(config.kernels, config.classes).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

  writer = SummaryWriter(log_dir = "./wandb/latest-run")
  total_batches = len(train_loader) * config.epochs
  example_ct = 0  # number of examples seen
  batch_ct = 0
  for epoch in tqdm(range(config.epochs)):
    for step, (images, labels) in enumerate(train_loader):
      images, labels = images.to(device), labels.to(device)
      # Forward pass ➡
      outputs = model(images)
      loss = criterion(outputs, labels)
      # Backward pass ⬅
      optimizer.zero_grad()
      loss.backward()
      # Step with optimizer
      optimizer.step()
      example_ct +=  len(images)
      batch_ct += 1
      # Report metrics every 25th batch
      if ((batch_ct + 1) % 25) == 0:
        writer.add_scalar("Train Metrics/loss", loss, batch_ct)
        writer.add_scalar("epoch", loss, batch_ct)
        logger.info(f"Epoch: {epoch}, Loss: {loss.detach().numpy()}")
    with torch.no_grad():
      correct, total = 0, 0
      for images, labels in validation_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          writer.add_scalar("Validation Metrics/loss", loss, batch_ct)
          writer.add_scalar("epoch", epoch, batch_ct)
      logger.info(f"Epoch {epoch}, Accuracy of the model on the {total} test images: {100 * correct / total}%")
      writer.add_scalar("Validation Metrics/accuracy", correct/total, batch_ct)
      writer.add_scalar("epoch", epoch, batch_ct)

  torch.save(model.state_dict(), "model.pt")
  model_artifact = wandb.Artifact(name = "mnist-model", type = "model")
  model_artifact.add_file("model.pt")
  run.log_artifact(model_artifact)


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.6152558922767639
INFO:CNN-Logger:Epoch: 0, Loss: 0.37864214181900024
INFO:CNN-Logger:Epoch: 0, Loss: 0.33058860898017883
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 test images: 93.25%
INFO:CNN-Logger:Epoch: 1, Loss: 0.18225780129432678
INFO:CNN-Logger:Epoch: 1, Loss: 0.14611569046974182
INFO:CNN-Logger:Epoch: 1, Loss: 0.04811842739582062
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 test images: 94.85%
INFO:CNN-Logger:Epoch: 2, Loss: 0.15746846795082092
INFO:CNN-Logger:Epoch: 2, Loss: 0.030076010152697563
INFO:CNN-Logger:Epoch: 2, Loss: 0.12193747609853745
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 test images: 95.75%
INFO:CNN-Logger:Epoch: 3, Loss: 0.06308609992265701
INFO:CNN-Logger:Epoch: 3, Loss: 0.10632984340190887
INFO:CNN-Logger:Epoch: 3, Loss: 0.10202132165431976
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 test images: 96.0%
INFO:CNN-Logger:Epoch: 4, Loss: 0.049882564693689346
INFO:CNN

Train Metrics/loss,█▅▅▃▂▁▃▁▂▁▂▂
Validation Metrics/accuracy,▁▅█
Validation Metrics/loss,█▃▅▁
epoch,▂▂▂▁▁▁▁▃▁▁▁▆▁▁▁█
global_step,▁▂▂▂▃▃▄▄▅▅▆▆▆▇██
Train Metrics/loss,0.10202
Validation Metrics/accuracy,0.9575
Validation Metrics/loss,0.08218
epoch,3.0
global_step,316


## Test Data Evaluation

In [27]:
import pandas as pd
with wandb.init(project = project_name, entity = entity, job_type = "evaluation") as run:
  model_artifact = run.use_artifact(model_artifact.wait())
  ## instantiate the model if necessary
  # model_dir = model_artifact.download()
  # model = ConvNet(config.kernels, config.classes)
  # model.load_state_dict(torch.load(f"{model_dir}/model.pt"))
  run.use_artifact(f"{run.entity}/{run.project}/mnist-test-data:latest")
  ## same goes for the dataset
  test_loader = make_loader(test, batch_size=config.batch_size)

  model.eval()
  # Run the model on some test examples

  with torch.no_grad():
      correct, total = 0, 0
      total_loss = 0
      all_data = []
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)*labels.size(0)
          total_loss += loss
          wandb_images = []
          for image in images.numpy():
            temp = wandb.Image(image)
            wandb_images.append(temp) 
          scores = pd.DataFrame( outputs.numpy().tolist(), columns = [f"p{i}" for i in range(outputs.shape[1])]).to_dict(orient = "series")
          data = {"images":wandb_images, "predicted": predicted.numpy().tolist(), "labels": labels.numpy().tolist()}
          data = {**data, **scores}
          all_data.append(pd.DataFrame(data))
      import pandas as pd 
      df = pd.concat(all_data)
      wandb.log({"Predictions vs Actuals": wandb.Table(dataframe = df)})
      run.log({"Test Metrics/loss": total_loss / total, "Test Metrics/accuracy": correct / total})
      logger.info(f"Accuracy of the model on the {total} " +
            f"test images: {100 * correct / total}%")
          

INFO:CNN-Logger:Accuracy of the model on the 2000 test images: 96.8%


Test Metrics/accuracy,▁
Test Metrics/loss,▁
Test Metrics/accuracy,0.968
Test Metrics/loss,0.10459


## HPO with Optuna

In [28]:

# @wandbc.track_in_wandb()
# def objective(trial):
#     learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
#     batch_size = trial.suggest_categorical(name = "batch_size", choices = [128, 256])
#     config_standard = dict(
#       epochs=5,
#       classes=10,
#       kernels=[16, 32],
#       dataset="MNIST",
#       architecture="CNN",
#       learning_rate = learning_rate, 
#       batch_size = batch_size
#     )
#     wandb.config.update(config_standard)
#     x = trial.suggest_float("x", -10, 10)
#     return (x - 2) ** 2




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ExperimentalWarning: track_in_wandb is experimental (supported from v3.0.0). The interface can change in the future.
  """Entry point for launching an IPython kernel.


## Training Function

In [29]:
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback

In [30]:
wandb_kwargs = {"project": "my-optuna-project-v5"}
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, as_multirun=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  


In [31]:
@wandbc.track_in_wandb()
def train_func(trial): 

    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    batch_size = trial.suggest_categorical(name = "batch_size", choices = [128, 256])
    config_standard = dict(
      epochs=5,
      classes=10,
      kernels=[16, 32],
      dataset="MNIST",
      architecture="CNN",
    )
    wandb.config.update(config_standard)
  
    wandb.use_artifact(f"{entity}/{project_name}/mnist-training-data:latest")
    wandb.use_artifact(f"{entity}/{project_name}/mnist-validation-data:latest")

    train_loader = make_loader(train, batch_size=config.batch_size)
    validation_loader = make_loader(validation, batch_size=config.batch_size)

    model = ConvNet(config.kernels, config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

    total_batches = len(train_loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
      for _, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        # Forward pass ➡
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass ⬅
        optimizer.zero_grad()
        loss.backward()
        # Step with optimizer
        optimizer.step()
        example_ct +=  len(images)
        batch_ct += 1
        # Report metrics every 25th batch
        if ((batch_ct + 1) % 25) == 0:
          logger.info(f"Epoch: {epoch}, Loss: {loss.detach().numpy()}")
          wandb.log({ "Train Metrics/loss": loss, "epoch": epoch})
      with torch.no_grad():
        correct, total = 0, 0
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            wandb.log({"Validation Metrics/loss": loss, "epoch": epoch})
        logger.info(f"Epoch {epoch}, Accuracy of the model on the {total} validation images: {100 * correct / total}%")
        wandb.log({"Validation Metrics/accuracy": correct / total})

    torch.save(model.state_dict(), f"{wandb.run.id}-model.pt")
    model_artifact = wandb.Artifact(name = f"{wandb.run.id}-mnist-model", type = "model")
    model_artifact.add_file(f"{wandb.run.id}-model.pt")
    wandb.log_artifact(model_artifact)
    return loss


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ExperimentalWarning: track_in_wandb is experimental (supported from v3.0.0). The interface can change in the future.
  """Entry point for launching an IPython kernel.


In [34]:
study = optuna.create_study()
study.optimize(train_func, n_trials=10, callbacks=[wandbc])

[I 2022-10-24 16:04:39,584] A new study created in memory with name: no-name-fb829780-3530-4509-bc48-2372be956dbf


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.2202364057302475
INFO:CNN-Logger:Epoch: 0, Loss: 0.1907530277967453
INFO:CNN-Logger:Epoch: 0, Loss: 0.19717641174793243
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 validation images: 94.95%
INFO:CNN-Logger:Epoch: 1, Loss: 0.12388281524181366
INFO:CNN-Logger:Epoch: 1, Loss: 0.10667406767606735
INFO:CNN-Logger:Epoch: 1, Loss: 0.05271652340888977
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 validation images: 97.25%
INFO:CNN-Logger:Epoch: 2, Loss: 0.07366706430912018
INFO:CNN-Logger:Epoch: 2, Loss: 0.030108748003840446
INFO:CNN-Logger:Epoch: 2, Loss: 0.0902952179312706
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 validation images: 97.0%
INFO:CNN-Logger:Epoch: 3, Loss: 0.018616069108247757
INFO:CNN-Logger:Epoch: 3, Loss: 0.06598187983036041
INFO:CNN-Logger:Epoch: 3, Loss: 0.022139111533761024
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 validation images: 97.25%
INFO:CNN-Logger:Epoch: 4, Loss: 0.019

Train Metrics/loss,█▇▇▅▄▃▃▂▄▁▃▂▁▁▃
Validation Metrics/accuracy,▁█▇█▇
Validation Metrics/loss,▅█▆▄▄▇▄▃▂▃▃▃▇▄▂▄▃▂▁▅▇▃▃▁▂▂▁▅▃▇▃▅▂▄▄▁▃▂▃▅
batch_size,▁
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
learning_rate,▁
trial_number,▁
value,▁
Train Metrics/loss,0.05613
Validation Metrics/accuracy,0.9705
Validation Metrics/loss,0.17458


  0%|          | 0/5 [00:00<?, ?it/s]

...

INFO:CNN-Logger:Epoch: 0, Loss: 0.3242708444595337
INFO:CNN-Logger:Epoch: 0, Loss: 0.2208414077758789
INFO:CNN-Logger:Epoch: 0, Loss: 0.14153040945529938
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 validation images: 95.85%
INFO:CNN-Logger:Epoch: 1, Loss: 0.2474503368139267
INFO:CNN-Logger:Epoch: 1, Loss: 0.15772059559822083
INFO:CNN-Logger:Epoch: 1, Loss: 0.11678603291511536
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 validation images: 96.55%
INFO:CNN-Logger:Epoch: 2, Loss: 0.07085666060447693
INFO:CNN-Logger:Epoch: 2, Loss: 0.016292011365294456
INFO:CNN-Logger:Epoch: 2, Loss: 0.04826488718390465
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 validation images: 96.5%
INFO:CNN-Logger:Epoch: 3, Loss: 0.020137270912528038
INFO:CNN-Logger:Epoch: 3, Loss: 0.09969673305749893
INFO:CNN-Logger:Epoch: 3, Loss: 0.03388528525829315
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 validation images: 97.0%
INFO:CNN-Logger:Epoch: 4, Loss: 0.03153

Train Metrics/loss,█▆▄▆▄▃▂▁▂▁▃▁▁▂▂
Validation Metrics/accuracy,▁▄▄▆█
Validation Metrics/loss,▅▂▄▃▇▅▂▄▃▅▃▄▁▃▃▂▂▂▄▃▃▃▃▆▂▃▁▄▄▂▄▃▁▃▃▃▂▁▄█
batch_size,▁
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
learning_rate,▁
trial_number,▁
value,▁
Train Metrics/loss,0.04008
Validation Metrics/accuracy,0.976
Validation Metrics/loss,0.35578


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.33528223633766174
INFO:CNN-Logger:Epoch: 0, Loss: 0.1397526115179062
INFO:CNN-Logger:Epoch: 0, Loss: 0.07320921123027802
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 validation images: 94.8%
INFO:CNN-Logger:Epoch: 1, Loss: 0.1139756292104721
INFO:CNN-Logger:Epoch: 1, Loss: 0.03201408311724663
INFO:CNN-Logger:Epoch: 1, Loss: 0.06809043884277344
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 validation images: 95.9%
INFO:CNN-Logger:Epoch: 2, Loss: 0.027186453342437744
INFO:CNN-Logger:Epoch: 2, Loss: 0.09580682963132858
INFO:CNN-Logger:Epoch: 2, Loss: 0.020073655992746353
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 validation images: 96.8%
INFO:CNN-Logger:Epoch: 3, Loss: 0.033711690455675125
INFO:CNN-Logger:Epoch: 3, Loss: 0.03615190088748932
INFO:CNN-Logger:Epoch: 3, Loss: 0.05919386073946953
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 validation images: 97.2%
INFO:CNN-Logger:Epoch: 4, Loss: 0.02731

Train Metrics/loss,█▄▂▃▁▂▁▃▁▁▂▂▁▁▂
Validation Metrics/accuracy,▁▄▇██
Validation Metrics/loss,▃▄▄▃▄▄█▂▄▂▂▃▂▂▃▅▂▂▂▃▃▃▃▇▅▂▁▃▁▂▂▃▂▂▃▂▂▃▁▃
batch_size,▁
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
learning_rate,▁
trial_number,▁
value,▁
Train Metrics/loss,0.04821
Validation Metrics/accuracy,0.9725
Validation Metrics/loss,0.09453


## Use API to interact with W&B

In [ ]:
import pandas as pd
import wandb
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")
temp_data = []
for r in sweep.runs:
 temp_dict = dict(**dict(r.summary), **r.config)
 temp_dict["run_id"] = r.id
 temp_dict["run_name"] = r.name
 temp_data.append( temp_dict)
df = pd.DataFrame(temp_data)
df.set_index("run_id", inplace = True)
best_run_id = sweep.best_run().id
best_run = api.run(f"{entity}/{project_name}/{best_run_id}")
df.loc[best_run_id]

wandb: Sorting runs by +summary_metrics.Validation Metrics/loss


_timestamp                     1663081318.589144
Train Metrics/loss                      0.254352
Validation Metrics/loss                 0.002262
Validation Metrics/accuracy                0.979
_step                                         99
epoch                                          4
_wandb                           {'runtime': 41}
_runtime                               42.174489
epochs                                         5
classes                                       10
dataset                                    MNIST
kernels                                 [16, 32]
batch_size                                   128
architecture                                 CNN
learning_rate                              0.005
run_name                         valiant-sweep-4
Name: 5o6kh8ef, dtype: object

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
project_name = "demos" #@param {type: "string"}
entity = "tim-w" #@param {type: "string"}


In [ ]:
import pandas as pd
import wandb
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")
temp_data = []
for r in sweep.runs:
 temp_dict = dict(**dict(r.summary), **r.config)
 temp_dict["run_id"] = r.id
 temp_dict["run_name"] = r.name
 temp_data.append( temp_dict)
df = pd.DataFrame(temp_data)
df.set_index("run_id", inplace = True)
best_run_id = sweep.best_run().id
best_run = api.run(f"{entity}/{project_name}/{best_run_id}")
df.loc[best_run_id]